In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/benjaminrike1/social_score_csr_reports.git

Cloning into 'social_score_csr_reports'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 259 (delta 14), reused 43 (delta 13), pack-reused 215
Receiving objects: 100% (259/259), 786.58 MiB | 8.19 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Checking out files: 100% (79/79), done.


In [4]:
pip install yfinance

     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 6.4 MB 11.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [49]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm

In [50]:
data_path = ''
social_scores = pd.read_csv(data_path+'social_scores.csv', index_col=0)

In [51]:
# reading in company-ticker mapping
df_financial = pd.read_csv('Financial_data/Tickers.csv', sep=';')
df_financial.head()

,Company,Ticker
0,3M,MMM
1,Abbott,ABT
2,Accenture,ACN
3,Adobe,ADBE
4,AEScorporation,AES


In [52]:
# merging financial and social_scores df
df_financial = df_financial.merge(social_scores, on='Company')
df_financial.head()

,Company,Ticker,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,total_2019,total_2020
0,3M,MMM,1745,1535,1806,1809,2096,7537,8099,8215,7996,9147
1,Abbott,ABT,1335,1913,918,1144,1602,5719,7288,5201,6317,11414
2,Accenture,ACN,822,919,802,865,210,2293,2526,1825,2570,435
3,Adobe,ADBE,80,99,124,185,222,409,499,616,818,940
4,AEScorporation,AES,2,165,524,518,199,117,1041,2125,3049,18734


In [53]:
#Adding Total Revenue and Market cap for each year
manual_data = pd.read_csv('Financial_data/Manual_finance_data.csv', sep=';')

df_financial = df_financial.merge(manual_data, on='Ticker')
df_financial.head()

,Company,Ticker,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,...,Total_revenue_2020,Total_revenue_2021,Market_cap_2017,Market_cap_2018,Market_cap_2019,Market_cap_2020,Market_cap_2021,Market_cap_2022,Unnamed: 13,Unnamed: 14
0,3M,MMM,1745,1535,1806,1809,2096,7537,8099,8215,...,32184,35355,107404,140188,110949,101450,100823,102360,NaN,NaN
1,Abbott,ABT,1335,1913,918,1144,1602,5719,7288,5201,...,34608,43075,60674,99316,127036,153608,187600,241365,NaN,NaN
2,Accenture,ACN,822,919,802,865,210,2293,2526,1825,...,44327,50533,74083,90984,105120,127749,157815,225748,NaN,NaN
3,Adobe,ADBE,80,99,124,185,222,409,499,616,...,12868,15785,49589,88493,122468,149837,228840,293345,NaN,NaN
4,AEScorporation,AES,2,165,524,518,199,117,1041,2125,...,9660,11141,7660,7152,9577,13211,15631,16201,NaN,NaN


In [54]:

#Calculating Revnue growth from total revenue
df_financial['Revenue_growth_2017'] = (df_financial['Total_revenue_2017']/df_financial['Total_revenue_2016'])-1
df_financial['Revenue_growth_2018'] = (df_financial['Total_revenue_2018']/df_financial['Total_revenue_2017'])-1
df_financial['Revenue_growth_2019'] = (df_financial['Total_revenue_2019']/df_financial['Total_revenue_2018'])-1
df_financial['Revenue_growth_2020'] = (df_financial['Total_revenue_2020']/df_financial['Total_revenue_2019'])-1
df_financial['Revenue_growth_2021'] = (df_financial['Total_revenue_2021']/df_financial['Total_revenue_2020'])-1

# Calculating average Market cap for each year (Market_Cap_2017 is from 01.01.2017 etc.)
df_financial['Avg_market_cap_2017'] = (df_financial['Market_cap_2017']+df_financial['Market_cap_2018'])/2
df_financial['Avg_market_cap_2018'] = (df_financial['Market_cap_2018']+df_financial['Market_cap_2019'])/2
df_financial['Avg_market_cap_2019'] = (df_financial['Market_cap_2019']+df_financial['Market_cap_2020'])/2
df_financial['Avg_market_cap_2020'] = (df_financial['Market_cap_2020']+df_financial['Market_cap_2021'])/2
df_financial['Avg_market_cap_2021'] = (df_financial['Market_cap_2021']+df_financial['Market_cap_2022'])/2

df_financial.head()

,Company,Ticker,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,...,Revenue_growth_2017,Revenue_growth_2018,Revenue_growth_2019,Revenue_growth_2020,Revenue_growth_2021,Avg_market_cap_2017,Avg_market_cap_2018,Avg_market_cap_2019,Avg_market_cap_2020,Avg_market_cap_2021
0,3M,MMM,1745,1535,1806,1809,2096,7537,8099,8215,...,0.051413,0.035000,-0.019197,0.001494,0.098527,123796.0,125568.5,106199.5,101136.5,101591.5
1,Abbott,ABT,1335,1913,918,1144,1602,5719,7288,5201,...,0.313480,0.116393,0.043365,0.084754,0.244654,79995.0,113176.0,140322.0,170604.0,214482.5
2,Accenture,ACN,822,919,802,865,210,2293,2526,1825,...,0.039630,0.133127,0.054230,0.025732,0.140005,82533.5,98052.0,116434.5,142782.0,191781.5
3,Adobe,ADBE,80,99,124,185,222,409,499,616,...,0.307486,0.236817,0.237099,0.151911,0.226686,69041.0,105480.5,136152.5,189338.5,261092.5
4,AEScorporation,AES,2,165,524,518,199,117,1041,2125,...,0.024219,0.019563,-0.050950,-0.051919,0.153313,7406.0,8364.5,11394.0,14421.0,15916.0


In [55]:
# extracting adujsted price for each year
df_financial['adjusted_01.01.2017'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2017-01-01',progress=False)['Adj Close'].values[0])
df_financial['adjusted_01.01.2018'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2018-01-01',progress=False)['Adj Close'].values[0])
df_financial['adjusted_01.01.2019'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2019-01-01',progress=False)['Adj Close'].values[0])
df_financial['adjusted_01.01.2020'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2020-01-01',progress=False)['Adj Close'].values[0])
df_financial['adjusted_01.01.2021'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2021-01-01',progress=False)['Adj Close'].values[0])
df_financial['adjusted_01.01.2022'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2022-01-01',progress=False)['Adj Close'].values[0])

In [56]:
df_financial['Exchange_ticker'] = '^GSPC'
df_financial.head()

,Company,Ticker,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,...,Avg_market_cap_2019,Avg_market_cap_2020,Avg_market_cap_2021,adjusted_01.01.2017,adjusted_01.01.2018,adjusted_01.01.2019,adjusted_01.01.2020,adjusted_01.01.2021,adjusted_01.01.2022,Exchange_ticker
0,3M,MMM,1745,1535,1806,1809,2096,7537,8099,8215,...,106199.5,101136.5,101591.5,150.212112,203.466705,168.888855,161.642197,167.790604,175.929382,^GSPC
1,Abbott,ABT,1335,1913,918,1144,1602,5719,7288,5201,...,140322.0,170604.0,214482.5,35.474598,54.646278,68.464058,83.581100,107.444374,140.245331,^GSPC
2,Accenture,ACN,822,919,802,865,210,2293,2526,1825,...,116434.5,142782.0,191781.5,107.060608,144.274887,134.628891,203.577194,257.353516,413.477997,^GSPC
3,Adobe,ADBE,80,99,124,185,222,409,499,616,...,136152.5,189338.5,261092.5,103.480003,177.699997,226.240005,329.809998,500.119995,567.059998,^GSPC
4,AEScorporation,AES,2,165,524,518,199,117,1041,2125,...,11394.0,14421.0,15916.0,9.654815,9.406152,13.016394,18.504044,22.773558,24.126663,^GSPC


In [57]:
df_financial['Exchange_01.01.2017'] = df_financial['Exchange_ticker'].apply(lambda x: yf.download(x, '2017-01-01',progress=False).Close.values[0])
df_financial['Exchange_01.01.2018'] = df_financial['Exchange_ticker'].apply(lambda x: yf.download(x, '2018-01-01',progress=False).Close.values[0])
df_financial['Exchange_01.01.2019'] = df_financial['Exchange_ticker'].apply(lambda x: yf.download(x, '2019-01-01',progress=False).Close.values[0])
df_financial['Exchange_01.01.2020'] = df_financial['Exchange_ticker'].apply(lambda x: yf.download(x, '2020-01-01',progress=False).Close.values[0])
df_financial['Exchange_01.01.2021'] = df_financial['Exchange_ticker'].apply(lambda x: yf.download(x, '2021-01-01',progress=False).Close.values[0])
df_financial['Exchange_01.01.2022'] = df_financial['Exchange_ticker'].apply(lambda x: yf.download(x, '2022-01-01',progress=False).Close.values[0])

df_financial['Exchangereturn_2017'] = df_financial['Exchange_01.01.2018']/df_financial['Exchange_01.01.2017']-1
df_financial['Exchangereturn_2018'] = df_financial['Exchange_01.01.2019']/df_financial['Exchange_01.01.2018']-1
df_financial['Exchangereturn_2019'] = df_financial['Exchange_01.01.2020']/df_financial['Exchange_01.01.2019']-1
df_financial['Exchangereturn_2020'] = df_financial['Exchange_01.01.2021']/df_financial['Exchange_01.01.2020']-1
df_financial['Exchangereturn_2021'] = df_financial['Exchange_01.01.2022']/df_financial['Exchange_01.01.2021']-1

In [58]:
df_financial

,Company,Ticker,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,...,Exchange_01.01.2018,Exchange_01.01.2019,Exchange_01.01.2020,Exchange_01.01.2021,Exchange_01.01.2022,Exchangereturn_2017,Exchangereturn_2018,Exchangereturn_2019,Exchangereturn_2020,Exchangereturn_2021
0,3M,MMM,1745,1535,1806,1809,2096,7537,8099,8215,...,2695.810059,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927
1,Abbott,ABT,1335,1913,918,1144,1602,5719,7288,5201,...,2695.810059,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927
2,Accenture,ACN,822,919,802,865,210,2293,2526,1825,...,2695.810059,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927
3,Adobe,ADBE,80,99,124,185,222,409,499,616,...,2695.810059,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927
4,AEScorporation,AES,2,165,524,518,199,117,1041,2125,...,2695.810059,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,WellsFargo,WFC,126,503,475,90,957,384,1596,1280,...,2695.810059,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927
96,Whirlpool,WHR,337,432,423,680,1096,1626,1654,2070,...,2695.810059,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927
97,Wynn,WYNN,111,467,417,464,611,404,1468,1826,...,2695.810059,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927
98,Xylem,XYL,556,632,636,999,1230,2550,2792,2647,...,2695.810059,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927


In [59]:
df_financial.to_csv('df_financial.csv')

In [60]:
df_financial['constant'] = [1 for i in range(df_financial.shape[0])]
df_financial.head()

,Company,Ticker,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,...,Exchange_01.01.2019,Exchange_01.01.2020,Exchange_01.01.2021,Exchange_01.01.2022,Exchangereturn_2017,Exchangereturn_2018,Exchangereturn_2019,Exchangereturn_2020,Exchangereturn_2021,constant
0,3M,MMM,1745,1535,1806,1809,2096,7537,8099,8215,...,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927,1
1,Abbott,ABT,1335,1913,918,1144,1602,5719,7288,5201,...,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927,1
2,Accenture,ACN,822,919,802,865,210,2293,2526,1825,...,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927,1
3,Adobe,ADBE,80,99,124,185,222,409,499,616,...,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927,1
4,AEScorporation,AES,2,165,524,518,199,117,1041,2125,...,2506.850098,3230.780029,3756.070068,4766.180176,0.193983,-0.070094,0.288781,0.162589,0.268927,1
